In [12]:
import numpy as np
import os #sistema operacional
import PIL #manipulação de imagem
import torch #aprendizado de maquina
import pandas as pd #manipular tabelas
import cv2

In [3]:
rotulo = pd.read_csv('./../SkinCancerClassifier/dados/rotulos')
rotulo

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern
...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,vidir_modern


In [ ]:
import os
import shutil
import pandas as pd

# Caminho da pasta com as imagens
pasta_imagens = './../SkinCancerClassifier/dataverse_files/images/'

# Ler a tabela com as informações "image_id" e "dx"
tabela = rotulo

# Iterar sobre cada imagem na pasta
for nome_imagem in os.listdir(pasta_imagens):
    # Extrair o image_id da imagem (supondo que o nome é no formato "image_id.jpg")
    image_id = nome_imagem.split('.')[0]

    # Verificar se o image_id existe na tabela
    if image_id in tabela['image_id'].values:
        # Localizar a categoria "dx" na tabela
        categoria = tabela.loc[tabela['image_id'] == image_id, 'dx'].values[0]

        # Criar a pasta com o nome da categoria, caso ainda não exista
        pasta_categoria = os.path.join(pasta_imagens, categoria)
        os.makedirs(pasta_categoria, exist_ok=True)

        # Mover a imagem para a pasta correspondente à categoria
        caminho_imagem = os.path.join(pasta_imagens, nome_imagem)
        shutil.move(caminho_imagem, os.path.join(pasta_categoria, nome_imagem))
    else:
        print(f"Image_id '{image_id}' não encontrado na tabela. A imagem '{nome_imagem}' não foi movida.")

In [ ]:
def get_filenames(base_path, sub_folder):
    # objetivo da funcao: obter todos os caminhos dos arquivos dentro
    # de "base_path/sub_folder/classification/"
    # base_path: caminho absoluto, SPLIT: treino/teste/validacao, classificacao: pneu/tuber/normal
    
    # concatencao do caminho, strings
    path = os.path.join(base_path, sub_folder)

    # para cada x em lista de arquivos no diretorio cujo caminho é path, concatenar
    # ex: C:\\Users\\Arthur\\Pictures\\DataSets\\trabalho... + img.jpg
    path_list = ([os.path.join(path,x) for x in os.listdir(path)])
    # retorna caminho das imagens para leitura
    return path_list

def create_pandas_cols(base_path, sub_folder, classification):
    # cria coluna para lista de arquivos e sua classe
    
    # cria uma tabela pandas DataFrame vazia
    df = pd.DataFrame()
    # cria uma coluna chamada path e atribui o resultado de get_filenames (funcao)
    df["path"] = get_filenames(base_path, sub_folder)
    # cria uma coluna com a constante classification
    df["class"] = classification
    return df

def create_pandas_daframe(base_path, sub_folder):
    # Vai concatenar a base de caminha de imagens de pessoas saudaveis(NORMAL) e doentes(sub_folder.upper)
    
    # criar uma base pandas com as imagens de pessoas saudaveis
#     dnormal = create_pandas_cols(base_path, sub_folder, split, "NORMAL")
    # cria uma base pandas com as imagens de pessoas doentes (restante = sub_folder (pneu /tuber))
    ddoente = create_pandas_cols(base_path, sub_folder, sub_folder.upper())
    # concatena no eixo x= 0 a base de normal e doentes UNION
    df = ddoente
#     df = pd.concat([dnormal,ddoente], axis=0, ignore_index=True)
    return df

### PRE PROCESSAMENTO

In [13]:
base_path = "/home/hub/Residencia/MBA/SkinCancerClassifier/dados/" #caminho
sub_folders = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

split = "train"

In [14]:
def get_filenames(base_path, sub_folders):
    
    path = os.path.join(base_path, sub_folders)
    path_list = ([os.path.join(path,x) for x in os.listdir(path)])
    return path_list

def create_pandas_cols(base_path, sub_folder, classification):
    df = pd.DataFrame()
    df["path"] = get_filenames(base_path, sub_folder)
    df["class"] = classification
    return df

def create_pandas_daframe(base_path, sub_folder):
    skins = create_pandas_cols(base_path, sub_folder, sub_folder.upper())
    df = skins
    return df

In [15]:
df_akiec = create_pandas_daframe(base_path, sub_folders[0])
df_bcc = create_pandas_daframe(base_path, sub_folders[1])
df_bkl = create_pandas_daframe(base_path, sub_folders[2])
df_df = create_pandas_daframe(base_path, sub_folders[3])
df_mel = create_pandas_daframe(base_path, sub_folders[4])
df_nv = create_pandas_daframe(base_path, sub_folders[5])
df_vasc = create_pandas_daframe(base_path, sub_folders[6])

In [16]:
df_doencas = pd.concat([df_akiec, df_bcc, df_bkl, df_df, df_mel, df_nv, df_vasc], ignore_index = True)
df_doencas

,path,class
0,/home/hub/Residencia/MBA/SkinCancerClassifier/...,AKIEC
1,/home/hub/Residencia/MBA/SkinCancerClassifier/...,AKIEC
2,/home/hub/Residencia/MBA/SkinCancerClassifier/...,AKIEC
3,/home/hub/Residencia/MBA/SkinCancerClassifier/...,AKIEC
4,/home/hub/Residencia/MBA/SkinCancerClassifier/...,AKIEC
...,...,...
13073,/home/hub/Residencia/MBA/SkinCancerClassifier/...,VASC
13074,/home/hub/Residencia/MBA/SkinCancerClassifier/...,VASC
13075,/home/hub/Residencia/MBA/SkinCancerClassifier/...,VASC
13076,/home/hub/Residencia/MBA/SkinCancerClassifier/...,VASC


In [17]:
def classify(a):
    if (a).lower() == "akiec":
        return -3
    if (a).lower() == "bcc":
        return -2
    if (a).lower() == "bkl":
        return -1
    if (a).lower() == "df":
        return 0
    if (a).lower() == "mel":
        return 1
    if (a).lower() == "nv":
        return 2
    if (a).lower() == "vasc":
        return 3

df_doencas["y"] = df_doencas["class"].apply(classify)

In [18]:
df_doencas.groupby('y').count()

,path,class
y,,
-3,987,987
-2,1028,1028
-1,1099,1099
0,610,610
1,1567,1567
2,7077,7077
3,710,710


In [19]:
from sklearn.model_selection import train_test_split

df_ttrain = pd.DataFrame()
df_test = pd.DataFrame()

X_train, X_test, y_train, y_test = train_test_split(
    df_doencas["path"], 
    df_doencas["y"], 
    test_size=0.30, 
    random_state=42,
    stratify=df_doencas["y"]
)

df_ttrain["path"] = X_train
df_ttrain["y"] = y_train

df_test["path"] = X_test
df_test["y"] = y_test

In [20]:
df_doencas.groupby('y').count()

,path,class
y,,
-3,987,987
-2,1028,1028
-1,1099,1099
0,610,610
1,1567,1567
2,7077,7077
3,710,710


In [21]:
from sklearn.model_selection import train_test_split

df_ttrain = pd.DataFrame()
df_test = pd.DataFrame()

X_train, X_test, y_train, y_test = train_test_split(
    df_doencas["path"], 
    df_doencas["y"], 
    test_size=0.30, 
    random_state=42,
    stratify=df_doencas["y"]
)

df_ttrain["path"] = X_train
df_ttrain["y"] = y_train

df_test["path"] = X_test
df_test["y"] = y_test

In [22]:
from sklearn.model_selection import train_test_split

df_val = pd.DataFrame()
df_train = pd.DataFrame()

X_train, X_val, y_train, y_val = train_test_split(
    df_ttrain["path"], 
    df_ttrain["y"], 
    test_size=0.1666666, 
    random_state=42,
    stratify=df_ttrain["y"]
)

df_train["path"] = X_train
df_train["y"] = y_train

df_val["path"] = X_val
df_val["y"] = y_val

In [23]:
# resetou index e excluiu os valores nulos Nan
df_train = df_train.reset_index(drop=True).dropna()
df_test = df_test.reset_index(drop=True).dropna()
df_val = df_val.reset_index(drop=True).dropna()

In [24]:
print(len(df_train)/len(df_akiec))
print(len(df_test)/len(df_akiec))
print(len(df_val)/len(df_akiec))

7.728470111448835
3.975683890577508
1.5460992907801419


In [25]:
df_train['split'] = 'train'
df_test['split'] = 'test'
df_val['split'] = 'val'
df_splits = pd.concat([df_train, df_test, df_val], ignore_index = True)
df_splits

,path,y,split
0,/home/hub/Residencia/MBA/SkinCancerClassifier/...,-3,train
1,/home/hub/Residencia/MBA/SkinCancerClassifier/...,1,train
2,/home/hub/Residencia/MBA/SkinCancerClassifier/...,2,train
3,/home/hub/Residencia/MBA/SkinCancerClassifier/...,1,train
4,/home/hub/Residencia/MBA/SkinCancerClassifier/...,-2,train
...,...,...,...
13073,/home/hub/Residencia/MBA/SkinCancerClassifier/...,-2,val
13074,/home/hub/Residencia/MBA/SkinCancerClassifier/...,-3,val
13075,/home/hub/Residencia/MBA/SkinCancerClassifier/...,1,val
13076,/home/hub/Residencia/MBA/SkinCancerClassifier/...,-3,val


In [26]:
def resize_img(img):
    width = 224
    height = 224
    dim = (width ,height)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return img

In [40]:
for row in range(len(df_splits)):
    img = cv2.imread(df_splits['path'][row])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = resize_img(img)
    cv2.imwrite("/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/" +str(df_splits['split'][row]) +
                 "/" + str(df_splits['y'][row]) + "/" + str(row) + ".jpg", cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    print("/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/" +str(df_splits['split'][row]) 
          + "/" + str(df_splits['y'][row]) + "/" + str(row) + ".jpg")

/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/-3/0.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/1/1.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/2/2.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/1/3.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/-2/4.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/2/5.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/2/6.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/3/7.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/2/8.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/2/9.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/-3/10.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/2/11.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/2/12.jpg
/home/hub/Residencia/MBA/SkinCancerClassifier/SPLIT_IMG/train/1/13.jpg
/home/hub/Res

In [45]:
diretorio = '../SkinCancerClassifier/SPLIT_IMG/val'  # Substitua pelo caminho do seu diretório

pastas_e_arquivos = []

for pasta in os.listdir(diretorio):
    if os.path.isdir(os.path.join(diretorio, pasta)):
        num_arquivos = len(os.listdir(os.path.join(diretorio, pasta)))
        pastas_e_arquivos.append({'Pasta': pasta, 'Quantidade de Arquivos': num_arquivos})

df = pd.DataFrame(pastas_e_arquivos)

print(df)


  Pasta  Quantidade de Arquivos
0    -1                     128
1    -3                     115
2     2                     826
3    -2                     120
4     3                      83
5     1                     183
6     0                      71
